<a href="https://colab.research.google.com/github/SriCharan2705/Fuel-Efficiency-Prediction-Using-Machine-Learning/blob/main/My_Fuel_Efficiency_Predictor_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(url, names=column_names, na_values='?',
                          comment='\t', sep=' ', skipinitialspace=True)

# Drop rows with missing values
dataset = raw_dataset.dropna()

# Split features and labels
cfeatures = dataset.drop('MPG', axis=1)
labels = dataset['MPG']

# Normalize features using mean and standard deviation
normalized_features = (cfeatures - cfeatures.mean()) / cfeatures.std()

# Split the data into training and testing sets
train_features = normalized_features[:300]
test_features = normalized_features[300:]
train_labels = labels[:300]
test_labels = labels[300:]

# Define the model
model1 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(train_features.keys())]),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(1)
])

# Compile the model
optimizer = tf.keras.optimizers.RMSprop(0.001)
model1.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

# Implement early stopping and learning rate scheduler
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20)

# Train the model
Mhistory = model1.fit(
    train_features, train_labels,
    epochs=1000, validation_split=0.2, verbose=0,
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate the model
test_loss, test_mae, test_mse = model1.evaluate(test_features, test_labels)

# Predict fuel efficiency for a new car
new_car_features = pd.DataFrame([[4, 120.0, 100.0, 2500, 15.0, 82, 1]], columns=column_names[1:])
normalized_new_car_features = (new_car_features - cfeatures.mean()) / cfeatures.std()
predicted_fuel_efficiency = model1.predict(normalized_new_car_features)

# Clipping the prediction to the range of actual MPG values in the dataset
predicted_fuel_efficiency = np.clip(predicted_fuel_efficiency, labels.min(), labels.max())

# Output results
print("Test MAE:", test_mae)
print("Test MSE:", test_mse)
print("Predicted Fuel Efficiency:", predicted_fuel_efficiency[0][0])
print("Min MPG:", labels.min())
print("Max MPG:", labels.max())


1/1 [==============================] - 0s 99ms/step
Test MAE: 4.4735493659973145
Test MSE: 31.868606567382812
Predicted Fuel Efficiency: 31.158506
Min MPG: 9.0
Max MPG: 46.6
